In [1]:
import torch
from torchvision import models
import torch.nn as nn
from torch.utils import data
from torchvision import transforms
import pickle
from hack_utils import NUM_PTS, CROP_SIZE, TRAIN_SIZE
from hack_utils import ThousandLandmarksDataset, create_submission, ScaleMinSideToSize, CropCenter, TransformByKeys, restore_landmarks
from hack_train import predict
import os
import pandas as pd
import numpy as np
from torch.nn import functional as fnn
import cv2
from matplotlib import pyplot as plt

In [2]:
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2 * NUM_PTS, bias=True)

In [3]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [4]:
model_name = 'resnet50_dofit2_lr_1e-4_epoch_0'

In [5]:
model.load_state_dict(torch.load(f"{model_name}.pth"))

<All keys matched successfully>

In [6]:
device = torch.device("cuda: 0") if torch.cuda.is_available() else torch.device("cpu")

In [7]:
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [8]:
train_transforms = transforms.Compose([
        ScaleMinSideToSize((CROP_SIZE, CROP_SIZE)),
        CropCenter(CROP_SIZE),
        TransformByKeys(transforms.ToPILImage(), ("image",)),
        TransformByKeys(transforms.ToTensor(), ("image",)),
        TransformByKeys(transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]), ("image",)),
    ])

# Test prediction

In [10]:
test_dataset = ThousandLandmarksDataset(os.path.join(os.path.abspath("data"), 'test'),
                                            train_transforms,
                                            split="test")
test_dataloader = data.DataLoader(test_dataset,
                                  batch_size=512,
                                  num_workers=4,
                                  pin_memory=True,
                                  shuffle=False,
                                  drop_last=False)

In [13]:
test_predictions = predict(model, test_dataloader, device)


test prediction...:  33%|████████████████████▎                                        | 65/195 [00:39<01:01,  2.12it/s]


test prediction...:  67%|████████████████████████████████████████▎                   | 131/195 [01:10<00:30,  2.09it/s]


test prediction...: 100%|████████████████████████████████████████████████████████████| 195/195 [01:41<00:00,  1.92it/s]


In [14]:
with open(f"{model_name}_tmp_test_predictions.pkl", "wb") as fp:
    pickle.dump({"image_names": test_dataset.image_names,
                 "landmarks": test_predictions}, fp)

In [16]:
create_submission(os.path.abspath("data"), test_predictions, f"{model_name}_submit.csv")